In [1]:
## import modules
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import re
import csv
import numpy as np

In [2]:
#Read review_train.json file
rv_train = []
with open('review_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    rv_tra = json.loads(line)
                    rv_train.append(rv_tra)
                    #print(rv_train)
                else:
                    break
        except:
            f.close()

In [287]:
#Read business_train.json file
bs_train = []
with open('business_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    bs_tra = json.loads(line)
                    bs_train.append(bs_tra)
                    #print(bs_train)
                else:
                    break
        except:
            f.close()

In [288]:
# merge business_train.json
 # change the format to dataframe
bs_train_df = pd.DataFrame(bs_train)

In [289]:
attr_bs = pd.read_csv('atrr_merge.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [290]:
business_train = pd.concat([bs_train_df,attr_bs],axis=1)
business_train = business_train.drop(["attributes"],axis=1)

In [291]:
category = business_train[["categories"]]
pat_rest = re.compile(r'.*Gym.*')
num_no_rest = []
for row in tqdm(range(len(category))):
    if category.iloc[row]['categories'] is None:
        num_no_rest.append(row)
    else:
        if not pat_rest.match(category.iloc[row]['categories']):
            num_no_rest.append(row)
business_train = business_train.drop(index = num_no_rest)

100%|██████████| 154606/154606 [00:25<00:00, 6175.43it/s]


In [292]:
business_train.columns

Index(['business_id', 'categories', 'city', 'hours', 'is_open', 'latitude',
       'longitude', 'name', 'postal_code', 'state', 'Unnamed: 0',
       'AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'Ambience', 'BYOB',
       'BYOBCorkage', 'BestNights', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly',
       'Caters', 'CoatCheck', 'Corkage', 'DietaryRestrictions', 'DogsAllowed',
       'DriveThru', 'GoodForDancing', 'GoodForKids', 'GoodForMeal',
       'HairSpecializesIn', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel',
       'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire',
       'RestaurantsCounterService', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi'],
      dtype='object')

In [293]:
names = ['AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'Ambience', 'BYOB',
       'BYOBCorkage', 'BestNights', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly',
       'Caters', 'CoatCheck', 'Corkage', 'DietaryRestrictions', 'DogsAllowed',
       'DriveThru', 'GoodForDancing', 'GoodForKids', 'GoodForMeal',
       'HairSpecializesIn', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel',
       'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire',
       'RestaurantsCounterService', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi']

In [294]:
#Print the attributes with missing values less than 40%
N = 0.4*len(business_train)
ind_nan = []
for i in range(len(names)):  
    if sum(pd.isnull(business_train[names[i]])) < N:
       print(names[i]) 
    else:
       ind_nan.append(names[i])  #Just keep the attributes:BikeParking, BusinessParking, ByAppointmentOnly, GoodForKids

BikeParking
BusinessParking
ByAppointmentOnly
GoodForKids


In [295]:
business_train = business_train.drop(ind_nan,axis = 1)
business_train = business_train.drop(['Unnamed: 0'],axis = 1)

In [296]:
#business_train.to_csv("business_train.csv" ,index = 0)

In [297]:
id = business_train['business_id']
id = id.as_matrix()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [298]:
#Read review_train.json file
rv_train = []
with open('review_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    rv_tra = json.loads(line)
                    rv_train.append(rv_tra)
                    #print(rv_train)
                else:
                    break
        except:
            f.close()
rv_train_df = pd.DataFrame(rv_train) # change the format to dataframe

In [299]:
id_rw = rv_train_df['business_id']
id_rw = id_rw.as_matrix()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [300]:
index = []
for num in range(len(id_rw)):
    if id_rw[num] in id:
        index.append(num)

In [301]:
#arr = range(len(rv_train_df))
#idx = np.delete(arr,index)
#review_train = rv_train_df.drop(index = idx)
review_train

,business_id,date,stars,text
0,31292,2013-05-07 04:34:36,1.0,Total bill for this horrible service? Over $8G...
238,104374,2015-05-26 07:01:54,4.0,This review is for just the gym facilities as ...
266,170000,2015-12-28 05:52:45,2.0,There are better and worse Y's for sure. This...
448,51689,2013-12-28 22:45:53,1.0,"Terrible Gym, the ceiling in the mens shower r..."
724,69171,2013-04-10 17:15:32,5.0,Fitness together\n\nI can say nothing but posi...
790,178332,2014-09-25 08:55:35,3.0,This Gold's is a pretty good gym..for the pric...
815,51630,2010-08-21 06:53:33,5.0,Wow...no reviews on here yet? I am surprised.....
945,53385,2015-06-08 18:21:10,5.0,I love this place! I started my Bootcamp worko...
955,151835,2014-10-11 05:55:38,4.0,So far so good...19.99 gets you and a guest i...
1116,61726,2018-01-21 14:54:04,3.0,GoodLife member for 3 years now... I personall...


In [239]:
#review_train.to_csv("review_train.csv",index = 0)

In [286]:
bs_pk = list(dict(business_train['BusinessParking']).values())

In [283]:
data = ({'garage': np.nan,
         'street': np.nan, 
         'validated': np.nan, 
         'lot': np.nan, 
         'valet':np.nan})
bs_parking = pd.DataFrame(data,index = [0])
for num in range(1,len(bs_pk)):
    try:
        bs_parking = pd.concat([bs_parking, pd.DataFrame(eval(bs_pk[num]), index = [num])])
        bs_parking[num:(num+1)] = (bs_parking[num:(num+1)] == 1.0)
    except:
        bs_parking = pd.concat([bs_parking, pd.DataFrame(data,index = [num])])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [310]:
test = business_train
test['garage_parking'] = bs_parking['garage']
test['lot_parking'] = bs_parking['lot']
test['street_parking'] = bs_parking['street']
test['valet_parking'] = bs_parking['valet']
test['validated_parking'] = bs_parking['validated']
#garage,lot,street,valet,validated

In [313]:
business_train = test.drop(['BusinessParking'],axis = 1)

In [314]:
business_train

,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,state,BikeParking,ByAppointmentOnly,GoodForKids,garage_parking,lot_parking,street_parking,valet_parking,validated_parking
16,104170,"Trainers, Health & Medical, Active Life, Physi...",Toronto,"{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",1,43.645041,-79.395799,Totum Life Science,M5V 1K4,ON,NaN,False,False,False,True,False,False,False
397,49708,"Active Life, Trainers, Fitness & Instruction, ...",North Las Vegas,"{'Monday': '15:30-19:30', 'Tuesday': '15:30-19...",1,36.276098,-115.203534,Iron Talon Crossfit,89084,NV,NaN,NaN,NaN,False,False,True,False,False
441,173815,"Gyms, Fitness & Instruction, Active Life",Glendale,"{'Monday': '17:30-21:0', 'Tuesday': '17:30-21:...",1,33.520077,-112.184332,Relentless Power Gym,85301,AZ,NaN,False,True,False,False,False,False,False
494,183154,"Fitness & Instruction, Gyms, Active Life",Toronto,"{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",1,43.638122,-79.418156,F45 Training Liberty Village,M6K 3E7,ON,True,False,False,False,True,False,False,False
568,138163,"Gyms, Active Life, Fitness & Instruction, Card...",Verdun,None,1,45.465383,-73.566886,Energie Cardio,H4G 1V1,QC,NaN,NaN,False,False,True,False,False,False
622,17594,"Active Life, Cardio Classes, Fitness & Instruc...",Las Vegas,"{'Monday': '4:30-21:30', 'Tuesday': '4:30-20:3...",1,36.298839,-115.279247,Orange Theory Fitness Las Vegas - Centennial,89117,NV,True,True,False,NaN,NaN,NaN,NaN,NaN
681,106407,"Taekwondo, Gyms, Martial Arts, Education, Acti...",Henderson,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,36.042930,-115.116226,WTM Taekwondo,89074,NV,False,False,NaN,False,True,False,False,False
710,170000,"Sports Clubs, Local Services, Community Servic...",Charlotte,"{'Monday': '5:0-21:30', 'Tuesday': '5:0-21:30'...",1,35.315808,-80.778680,University City Branch YMCA,28262,NC,True,False,True,NaN,NaN,NaN,NaN,NaN
713,69171,"Active Life, Gyms, Trainers, Fitness & Instruc...",Charlotte,"{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",1,35.101964,-80.991144,Fitness Together,28278,NC,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
801,22591,"Gyms, Trainers, Active Life, Fitness & Instruc...",Henderson,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.035840,-115.098688,24 Hour Fitness - Wigwam Henderson,89074,NV,True,False,True,False,False,False,False,False


In [315]:
business_train.to_csv("business_train_new.csv")